# Clustering ensemble

Grupowanie może być także używane jako preprocessing do klasyfikacji. Idea jest zbliżona do komitetów klasyfikatorów.

Proszę wziąć dane ecoli z UCI. Podzielić na zbiór testowy i treningowy.

1. Uczymy logistyczną regresję i testujemy (accuracy)

2.  
    * Na zbiorze treningowym robimy k-means (5 klastrów) co daje podział. 
    * W każdym klastrze uczymy logistyczną regresję. 
    * Testujemy przypisując punkt do najbliższego klastra, a następnie przewidując modelem klasyfikacyjnym z tego klastra.

Które podejście daje lepszy wynik. Czy drugie podejście radzi sobie z klasami nieseparowalnymi liniowo?

In [13]:
import pandas as pd
import numpy as np

df = pd.read_csv(filepath_or_buffer='./ecoli.dat',  header=None, sep=',')
df.head()

,0,1,2,3,4,5,6,7
0,49.0,29.0,48.0,5.0,56.0,24.0,35.0,cp
1,7.0,4.0,48.0,5.0,54.0,35.0,44.0,cp
2,56.0,4.0,48.0,5.0,49.0,37.0,46.0,cp
3,59.0,49.0,48.0,5.0,52.0,45.0,36.0,cp
4,23.0,32.0,48.0,5.0,55.0,25.0,35.0,cp


Zaprezentuj dane jako macierze:

   * X
   * y

zawierające odpowiednie dane i przydziały do grup.
  

In [14]:
X = np.array(df.iloc[:, :-1])
X
print(X.shape)
y = df.iloc[:,-1]

labels = set(y)
labels
#{cp,im,imS,imL,imU,om,omL,pp}

(336, 7)


{'cp', 'im', 'imL', 'imS', 'imU', 'om', 'omL', 'pp'}

In [15]:
z = {'cp':0, 'im':1, 'imL':2, 'imS':3, 'imU':4, 'om':5, 'omL':6, 'pp':7}
z

{'cp': 0, 'im': 1, 'imL': 2, 'imS': 3, 'imU': 4, 'om': 5, 'omL': 6, 'pp': 7}

In [16]:
y = np.array([z[key] for key in y])
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       3, 3, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7,
       7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,

In [18]:
Y=y

Podziel dane na train/test

In [19]:
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=4)
print(X_train.shape[0])
print(X_test.shape[0])

268
68


Wykonaj klastrowanie k-means z k = 5
 * naucz metodę na X_train
 * przydziel elementy z X_test do odpowiednich klastrów wykorzystując model nauczony na X_train

In [20]:
k = 5

from sklearn.cluster import KMeans

cluster = KMeans(n_clusters=k, random_state = 42)
# slice matrix so we only include the 0/1 indicator columns in the clustering
clust_train = cluster.fit_predict(X_train)
clust_test = cluster.predict(X_test)

In [42]:
np.unique(clust_train)

array([0, 1, 2, 3, 4])

Podziel dane treningowe i testowe na grupy.

In [54]:
#wyciagam dane z kolejnych klas
def split(X, y, clustering):
    data = []
    labels = []
    for j in range(len(np.unique(clustering))):
        data.append([X[i] for i in range(X.shape[0]) if clustering[i] == j])
        labels.append([y[i] for i in range(y.shape[0]) if clustering[i] == j])
   
    return(data, labels)     

In [57]:
data_train, labels_train = split(X_train, y_train, clust_train)
data_test, labels_test = split(X_test, y_test, clust_test)

Naucz regresję logistyczną w każdym zbiorze i policz accuracy_score

Sumaryczne 

$$
accuracy_score = len(X_1) * accuracy\_score(X_1) + ... + len(X_n) * accuracy\_score(X_n)
$$

In [58]:
from sklearn.metrics import accuracy_score

In [59]:
logreg = linear_model.LogisticRegression(C=1e5, random_state= 42) 
acc_te = 0
acc_tr = 0
for i in range(k):
    logreg.fit(data_train[i], labels_train[i])
    y_train_pred = logreg.predict(data_train[i])
    acc_train = accuracy_score(labels_train[i], y_train_pred)
    print("Dokładność na zbiorze treningowym ", acc_train)
    acc_tr = acc_tr + len(labels_train[i])*acc_train
    
    y_test_pred = logreg.predict(data_test[i])
    acc_test = accuracy_score(labels_test[i], y_test_pred)
    print("Dokładność na zbiorze treningowym ", acc_test)
    print("-------------------------------")
    acc_te = acc_te + len(labels_test[i])*acc_test
    
acc_tr = acc_tr/len(X_train)
acc_te = acc_te/len(X_test)

print("Dokładność na zbiorze treningowym ", acc_tr)
print("Dokładność na zbiorze test ", acc_te)


Dokładność na zbiorze treningowym  0.9223300970873787
Dokładność na zbiorze treningowym  1.0
-------------------------------
Dokładność na zbiorze treningowym  0.717391304347826
Dokładność na zbiorze treningowym  0.6666666666666666
-------------------------------
Dokładność na zbiorze treningowym  0.9122807017543859
Dokładność na zbiorze treningowym  0.8
-------------------------------
Dokładność na zbiorze treningowym  1.0
Dokładność na zbiorze treningowym  1.0
-------------------------------
Dokładność na zbiorze treningowym  1.0
Dokładność na zbiorze treningowym  0.7777777777777778
-------------------------------
Dokładność na zbiorze treningowym  0.9029850746268657
Dokładność na zbiorze test  0.8823529411764706


Naucz jeden model regresji na całym zbiorze

In [60]:
logreg = linear_model.LogisticRegression(C=1e5, random_state = 42) # w prakcy powinniśmy dopasować parametr c
logreg.fit(X_train, y_train)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=42,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [61]:
y_train_pred = logreg.predict(X_train)
print("Dokładność na zbiorze treningowym ", accuracy_score(y_train, y_train_pred))
y_test_pred = logreg.predict(X_test)
print("Dokładność na zbiorze test ", accuracy_score(y_test, y_test_pred))

Dokładność na zbiorze treningowym  0.7985074626865671
Dokładność na zbiorze test  0.8235294117647058
